# OpenAI API Test

In [28]:
import openai
import os
from dotenv import load_dotenv
import re

[nltk_data] Downloading package punkt to C:\Users\TEMP.BECAYES-
[nltk_data]     MATEBOO.000\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

## Configure the API Key

In [30]:
# Load environment variables from .env file
load_dotenv('secret.env')

# Access API key
api_key = os.getenv("OPENAI_API_KEY")

# Set API key
openai.api_key = api_key

## Chat Completion

See doc: [Chat completion](https://platform.openai.com/docs/api-reference/chat/create).

In [148]:
def trim_text(generated_text):
    """
    Trim the text.

    The text generated by the model is not always complete due
    to the max_token limit. It is very likely that the last
    sentence will be cut-off. This function removes the last
    sentence if it does not end with a punctuation.

    Note that the function assumes a sentence ends with ".",
    "!", "?", or '...'.
    """
    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?...])\s', generated_text)
    
    # Remove \n in the first sentences that may have been caused by new lines
    sentences[0] = sentences[0].lstrip('\n')

    # Remove the last sentence if it does not end with a punctuation
    if len(sentences) > 1 and not sentences[-1].endswith(('.','!','?' ,'...')):
        sentences = sentences[:-1]

    return ' '.join(sentences)

### Davinci

In [146]:
response = openai.Completion.create(
    engine = "text-davinci-003", 
    prompt = "Once upon a time",
    max_tokens = 15,  
    n = 1 
)

In [149]:
generated_text = response.choices[0].text
 
print(generated_text)
trimmed_text = trim_text(generated_text)
trimmed_text



once upon a time there was a kingdom filled with lush green fields


['once upon a time there was a kingdom filled with lush green fields']

### GPT 3.5

**Set up the conversation**

In [109]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant. Your response are extremly short and concise. You do not generate more than 50 tokens."},  # Optional system message
    {"role": "user", "content": "Who won the world series in 2020?"},
    # {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    # {"role": "user", "content": "Where was it played?"}
]

**Get a response**

In [110]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        max_tokens=100,
)

In [135]:
response.choices[0].message.content

**Pursue the conversation**

In [69]:
# User's next message
user_message = {"role": "user", "content": "Where was it played?"}

# Add the user's message to the ongoing conversation
conversation.append(user_message)

In [96]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=conversation,
    max_tokens=256,
)

In [ ]:
response

### Enable Streaming

The API can return text as partial message deltas, like in GPT.

In [120]:
response_chunks = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=conversation,
    max_tokens=50,
    stream=True  # Enable streaming
)

In [121]:
for chunk in response_chunks:
    if response.choices[0].finish_reason != "stop":
        print(response.choices[0].delta.content)


Los
 Angeles
 Dodgers
.


**Custom function for streamming**

The API way is not conveniant. Let's build our own function.

In [133]:
import time

def stream_text(text):
    for line in text.split(" "):
        print(line, end=" ", flush=True)
        time.sleep(0.1)

stream_text(generated_text)



there was a warrior who lived in a remote mountain village. He 

## Gradio App

In [140]:
import gradio as gr

d:\Natural Language Processing\GPT-test\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [141]:
conversation = [
    {"role": "system", "content": "You are a Machine Learning Engineer. You provide advices. Your response are extremly short and concise. You do not generate more than 50 tokens."},
]

def generate_text(prompt):
    """
    Takes in a prompt and returns a response from the model.
    The function keeps the conversation history in the global variable conversation.
    """  
    user_message = {"role": "user", "content": prompt}
    conversation.append(user_message)
 
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        max_tokens=100,
    )

    return response.choices[0].message.content

In [143]:
# Create a Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    title="Machine Learning Advisor",
    description="Ask for advices, our consultant will guide you."
)

# Launch the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
